# Day 12 Challenge

In [5]:
import aocd
import numpy as np
import datetime
from math import ceil
from numpy import abs

In [143]:
def convert(list): 
    res = "".join(map(str, list))
    return res 


class Decoder:
    
    def __init__(self, input_signal, pattern=[0,1,0,-1], signal_multiplikator=2):
        
        self._base_signal = list(map(int, list(input_signal*signal_multiplikator)))
        self._input_signal = list(map(int, list(input_signal*signal_multiplikator)))
        self._base_pattern = pattern
        
        self._input_offset = int(input_signal[:7])
        
        self._input_signal_length = len(self._input_signal)
        
        self._walk_path = []
    
    def get_pattern(self, multiplikator):
        
        pl = len(self._base_pattern)
        sl = len(self._input_signal)
        
        # start with an empty pattern
        pattern = []
        
        # negative or zero multiplikator is not possible
        assert multiplikator > 0, f"Multiplikator must be positive but was {multiplikator}."
        
        # take base pattern items
        for item in self._base_pattern * (ceil(sl/pl/multiplikator)+1):
            
            # use item n times (depending on multiplikator)
            for i in range(multiplikator):
                if len(pattern) <= len(self._input_signal)+1:
                    pattern.append(item)
        
        # return pattern but omit first item
        return pattern[1:]
    
    
    def step(self):
        
        step_output_list = []
        
        # make as many iterations as needed
        for iteration in range(1, self._input_signal_length+1):
            
            # get the pattern for this iteration
            pattern = self.get_pattern(iteration)
            
            i = 0
            
            iteration_sum = 0
            
            for i in range(len(self._input_signal)):
                iteration_sum += self._input_signal[i] * pattern[i]
            
            # write iteration sum to list
            step_output_list.append(int(str(iteration_sum)[-1]))
        return convert(step_output_list)

    
    def walk(self, iterations=100):
        for i in range(iterations):
            self._input_signal = list(map(int, list(self.step())))
            
    def get_result(self, part=1):
        if part == 1:
            return int(convert(self._input_signal[:8]))
        elif part == 2:
            return int(convert(self._input_signal[int(self._input_offset):int(self._input_offset+8)]))

## Testing

In [ ]:
data = "03036732577212944063491565474664"
dec = Decoder(input_signal=data, signal_multiplikator=10000)
dec._input_offset
dec.walk(100)
dec.get_result(part = 2)

## Part 1

In [146]:
session:str = "53616c7465645f5f5d64e0f6b2811f4d18eb862dae5aa906e4d25c0f6a1d5944c89433699f1b1175fdea140e981da4a5"
day = 16
year = 2019
data = aocd.get_data(session=session, year=year, day=day)

#dec = Decoder(input_signal=data, sig)
#dec.walk(100)
#dec.get_result()

## Part 2

In [147]:
from itertools import cycle, repeat, islice, count
from functools import partial,lru_cache
import numpy as np

# part1-2 numpy, too slow for part 2:
def day16(inp, phases=100):
    init = np.array(list(map(int, inp.strip())))
    dat = init
    ran = np.arange(dat.size)

    for phase in range(phases):
        out = np.empty_like(dat)
        for i in range(dat.size):
            # [0, 1, 0, -1]: ones = [1] + k*4 -> shift -1: [0] + k*4
            # [0, 0, 1, 1, 0, 0, -1, -1]: ones = [2, 3] + k*8 -> shift -1: [1, 2] + k*8
            # [0, 0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1]: ones = [3, 4, 5] + k*16 -> shift -1: [2, 3, 4] + k*12
            # [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, -1, -1, -1, -1]: ones = [4, 5, 6, 7] + k*16 -> shift -1: [3, 4, 5, 6] + k*16
            #
            # [0, 1, 0, -1]: minusones = [3] + k*4 -> shift -1: [2] + k*4
            # [0, 0, 1, 1, 0, 0, -1, -1]: minusones = [6, 7] + k*8 -> shift -1: [5, 6] + k*8
            # [0, 0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1]: minusones = [9, 10, 11] + k*16 -> shift -1: [8, 9, 10] + k*12

            ranmod = ran % (4*(i+1))
            ones = (i <= ranmod) & (ranmod <= 2*i)
            minusones = (3*(i+1) - 1 <= ranmod) & (ranmod <= 4*(i+1) - 2)
            out[i] = abs(dat[ones].sum() - dat[minusones].sum()) % 10
        dat = out

    res = ''.join(map(str, out[:8]))

    return res

def day16b(inp, phases=100):
    init = np.array(list(map(int, inp.strip())))
    dat = np.tile(init, 10000)

    # assume that the requested bits are always in the tail
    # where the mapping matrix is trivial (upper triangular with full ones)
    offset = int(''.join(map(str, init[:7])))
    half = dat.size//2
    assert offset >= half

    for phase in range(phases):
        out = np.empty_like(dat)
        out[-1:half-1:-1] = abs(dat[-1:half-1:-1].cumsum()) % 10
        dat = out

    res = ''.join(map(str, out[offset:offset+8]))

    return res

print(day16b(data))

19903864
